# Tutorial: Interactive Maps with Python and Folium, Part 1
## Basic Maps and Circle Markers
#### Using data from NYC CitiBike program


In [1]:
import pandas as pd
import folium

To test that everything is working, let’s pull up a map of New York City and add a circle marker.

In [2]:
folium_map = folium.Map(location=[40.738, -73.98],
                        zoom_start=13,
                        tiles="CartoDB dark_matter")

folium.CircleMarker(location=[40.738, -73.98],fill=True).add_to(folium_map)
folium_map

#### Showing some real data, NYC bike trips
Next, we’ll load some data. The NYC bike share program makes its data public, you can download it here:
https://www.citibikenyc.com/system-data.
Just one month of data will do for this example. 

We’ll use pandas to load the data into python, we’ll convert time strings into DateTime objects

In [27]:
bike_data = pd.read_csv(r'C:\users\matth\Desktop\202006-citibike-tripdata.csv')
bike_data["starttime"] = pd.to_datetime(bike_data["starttime"])
bike_data["stoptime"] = pd.to_datetime(bike_data["stoptime"])
bike_data["hour"] = bike_data["starttime"].map(lambda x: x.hour)

In [29]:
bike_data.tail()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender,hour
1882268,685,2020-06-30 23:59:41.116,2020-07-01 00:11:06.779,503,E 20 St & Park Ave,40.738274,-73.987520,3746,6 Ave & Broome St,40.724308,-74.004730,21056,Customer,1993,2,23
1882269,446,2020-06-30 23:59:46.426,2020-07-01 00:07:13.086,3699,W 50 St & 9 Ave,40.763605,-73.989180,523,W 38 St & 8 Ave,40.754666,-73.991382,30164,Customer,1969,0,23
1882270,439,2020-06-30 23:59:47.477,2020-07-01 00:07:06.559,3699,W 50 St & 9 Ave,40.763605,-73.989180,523,W 38 St & 8 Ave,40.754666,-73.991382,43742,Customer,1986,1,23
1882271,890,2020-06-30 23:59:53.395,2020-07-01 00:14:43.427,3852,Stewart Ave & Johnson Ave,40.708690,-73.925870,3054,Greene Ave & Throop Ave,40.689493,-73.942061,15787,Subscriber,1994,1,23
1882272,433,2020-06-30 23:59:53.901,2020-07-01 00:07:07.095,324,DeKalb Ave & Hudson Ave,40.689888,-73.981013,261,Johnson St & Gold St,40.694749,-73.983625,38767,Customer,1990,1,23


### pre-processing data
We'll write a function that does the following: 
- generate a DataFrame containing locations of stations
- generates a DataFrame containing the number of trips originating at each station. 
- generates a DataFrame containing the number of trips arriving at each station. 
- join the three dataframes into one.

In [30]:
def get_trip_counts_by_hour(selected_hour):
    # make a DataFrame with locations for each bike station
    locations = bike_data.groupby("start station id").first()
    locations = locations.loc[:, ["start station latitude",
                                 "start station longitude",
                                 "start station name"]]
    
    #select one time of day
    subset = bike_data[bike_data["hour"]==selected_hour]
    
    # count trips for each destination
    departure_counts =  subset.groupby("start station id").count()
    departure_counts = departure_counts.iloc[:,[0]]
    departure_counts.columns= ["departure count"]
    
    # count trips for each origin
    arrival_counts =  subset.groupby("end station id").count().iloc[:,[0]]
    arrival_counts.columns= ["arrival count"]

    #join departure counts, arrival counts, and locations
    trip_counts = departure_counts.join(locations).join(arrival_counts)
    return trip_counts

# print a sample to check our code works
get_trip_counts_by_hour(6).head()

,departure count,start station latitude,start station longitude,start station name,arrival count
start station id,,,,,
72,102,40.767272,-73.993929,W 52 St & 11 Ave,75.0
79,12,40.719116,-74.006667,Franklin St & W Broadway,29.0
82,20,40.711174,-74.000165,St James Pl & Pearl St,19.0
83,10,40.683826,-73.976323,Atlantic Ave & Fort Greene Pl,19.0
116,103,40.741776,-74.001497,W 17 St & 8 Ave,26.0


Now we'll write a function that generates a new folium map and adds circle markers for each station.

In [33]:
def plot_station_counts(trip_counts):
    # generate a new map
    folium_map = folium.Map(location=[40.738, -73.98],
                            zoom_start=13,
                            tiles="CartoDB dark_matter",
                            width='50%')

    # for each row in the data, add a cicle marker
    for index, row in trip_counts.iterrows():
        # calculate net departures
        net_departures = (row["departure count"]-row["arrival count"])
        
        # generate the popup message that is shown on click.
        popup_text = "{}<br> total departures: {}<br> total arrivals: {}<br> net departures: {}"
        popup_text = popup_text.format(row["start station name"],
                          row["arrival count"],
                          row["departure count"],
                          net_departures)
        
        # radius of circles
        radius = net_departures/20
        
        # choose the color of the marker
        if net_departures>0:
            # color="#FFCE00" # orange
            # color="#007849" # green
            color="#E37222" # tangerine
        else:
            # color="#0375B4" # blue
            # color="#FFCE00" # yellow            
            color="#0A8A9F" # teal
        
        # add marker to the map
        folium.CircleMarker(location=(row["start station latitude"],
                                      row["start station longitude"]),
                            radius=radius,
                            color=color,
                            popup=popup_text,
                            fill=True).add_to(folium_map)
    return folium_map

### Showing Real data
We'll make 2 maps to show the different patterns for bike migration at 9 am and 6pm.

In [34]:
# plot net departures at 9AM

trip_counts = get_trip_counts_by_hour(9)
plot_station_counts(trip_counts)

In [35]:
# plot net departures at 6PM

trip_counts = get_trip_counts_by_hour(18)
folium_map = plot_station_counts(trip_counts)
folium_map

In [9]:
folium_map.save("part_1.html")